In [ ]:
# crawl_lazada_listpage_rating_pages.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time, pandas as pd

# ---------- cấu hình Chrome ----------
opts = Options()
# opts.add_argument("--headless")   # uncomment nếu muốn chạy ẩn
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")
opts.add_argument("--disable-blink-features=AutomationControlled")
opts.add_experimental_option("excludeSwitches", ["enable-automation"])
opts.add_experimental_option('useAutomationExtension', False)
opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

urls = {
    "Điện thoại": "https://www.lazada.vn/catalog/?q=điện%20thoại",
    "Quần áo": "https://www.lazada.vn/catalog/?q=quần%20áo",
    "Tủ lạnh": "https://www.lazada.vn/catalog/?q=tủ%20lạnh",
    "Tivi": "https://www.lazada.vn/catalog/?q=ti%20vi"
}

FULL, HALF = "Dy1nx", "JhbDv"  # keyword nhận diện icon sao

def scroll(times=4, pause=2):
    last = drv.execute_script("return document.body.scrollHeight")
    for _ in range(times):
        drv.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        new = drv.execute_script("return document.body.scrollHeight")
        if new == last: break
        last = new

def parse_rating(p):
    stars = p.find_elements(By.CSS_SELECTOR, "i._9-ogB")
    if not stars: return "N/A"
    full = sum(FULL in s.get_attribute("class") for s in stars)
    half = sum(HALF in s.get_attribute("class") for s in stars)
    return full + 0.5 * half if (full or half) else "N/A"

def crawl_page(url, category, page_num):
    page_url = url + f"&page={page_num}"
    drv.get(page_url)
    time.sleep(3)
    scroll()

    products = drv.find_elements(By.CSS_SELECTOR, "div.Bm3ON")
    print(f"[{category}] Page {page_num} -> {len(products)} sản phẩm")
    rows = []
    for p in products:
        try:
            name = p.find_element(By.CSS_SELECTOR, "div.RfADt a").get_attribute("title")
        except: name = "N/A"

        try: price = p.find_element(By.CSS_SELECTOR, "span.ooOxS").text
        except: price = "N/A"

        try: sold = p.find_element(By.XPATH, ".//span[contains(text(),'Đã bán')]").text
        except: sold = "N/A"

        rating = parse_rating(p)

        rows.append({
            "Category": category,
            "Page": page_num,
            "Name": name,
            "Price": price,
            "SoldCount": sold,
            "Rating": rating
        })
    return rows

# ---------- crawl 3 trang mỗi category ----------
all_rows = []
for cat, url in urls.items():
    for page in range(1, 4):   # page 1 -> 3
        all_rows.extend(crawl_page(url, cat, page))

# ---------- lưu ----------
if all_rows:
    df = pd.DataFrame(all_rows)
    df.to_excel("lazada_products_pages.xlsx", index=False)
    print("✅ Lưu xong lazada_products_pages.xlsx")
else:
    print("❌ Không lấy được dữ liệu")


Đang crawl Điện thoại ...
[Điện thoại] tìm thấy 0 sản phẩm
Đang crawl Quần áo ...
[Quần áo] tìm thấy 0 sản phẩm
Đang crawl Tủ lạnh ...
[Tủ lạnh] tìm thấy 0 sản phẩm
Đang crawl Tivi ...
[Tivi] tìm thấy 0 sản phẩm
 Crawl xong! Dữ liệu lưu vào lazada_products.xlsx
